# Dependency Inversion Principle (DIP)

> High-level modules should not import anything from low-level modules; both should depend on abstractions

> Abstractions should not depend on details. Details (concrete implementations) should depend on abstractions.

(Do not confuse with dependency injection!)

Essentially, you want to depend on interfaces rather than concrete implementations, because that way you can swap one for the other.

For this example, we'll simulate genealogy research. Let's begin by creating the `Relationship` enum and `Person` class.

In [1]:
from enum import Enum

class Relationship(Enum):
    PARENT = 0
    CHILD = 1
    SIBLING = 2

class Person:
    def __init__(self, name):
        self.name = name

Let's also create a `Relationships` class that will store the relation between 2 people:

In [2]:
class Relationships():
    def __init__(self):
        self.relations = [] # we store relationships as a list, keep it in mind

    def add_parent_and_child(self, parent, child):
        self.relations.append((parent, Relationship.PARENT, child))
        self.relations.append((child, Relationship.CHILD, parent))

This is a *low-level module* because it's on the "first layer" and deals with our base data.

Let's now create a *higher-level* `Research` module that builds on top on `Relationships` and thus depends on it.

In [3]:
class Research:
    def __init__(self, relationships):
        relations = relationships.relations # we're copying the Relationships list to Research
        for r in relations:
            if r[0].name == 'John' and r[1] == Relationship.PARENT:
                print(f'John has a child called {r[2].name}.')

Let's test our code:

In [4]:
parent = Person('John')
child1 = Person('Chris')
child2 = Person('Matt')

relationships = Relationships()
relationships.add_parent_and_child(parent, child1)
relationships.add_parent_and_child(parent, child2)

Research(relationships)

John has a child called Chris.
John has a child called Matt.


The code seems to work fine, but there's a big issue: we depend on how `Relationships` stores the relations, which is currently a list.

It's possible that in the future we may need to change from a list to a more complex data structure for more sophisticated use cases. However, by doing so, we would break `Research`. Our current code is breaking DIP, because we're accessing the internal storage mechanism of a low-level module in our high-level module.

We can restore DIP by defining an interface for the low-level module. `Research` would then depend on this interface (abstraction) rather than the implementation itself.

We will now create this interface:

In [5]:
from abc import ABC, abstractmethod

class RelationshipBrowser(ABC):
    @abstractmethod
    def find_all_children_of(self, name): pass

The interface provides a method to search relationships, which was previously implemented in `Research`. We have moved that functionality to a lower level, which means that `Research` no longer depends on how the relationships are implemented and we can update `Relationships` with a different implementation for storage without worrying about breaking higher-level modules.

Let's update the `Relationships` class with an implementation of the interface:

In [6]:
class Relationships(RelationshipBrowser):
    relations = []

    def add_parent_and_child(self, parent, child):
        self.relations.append((parent, Relationship.PARENT, child))
        self.relations.append((child, Relationship.PARENT, parent))
            
    def find_all_children_of(self, name):
        """
        'yield' works like 'return' but does not stop execution of the current code block.
        It's very useful for methods that return lists without having to construct the list itself within the method and then returning the complete list, because we can return each list element as we obtain it.
        This method will return a list with all the children of the 'name' parent.
        """
        for r in self.relations:
            if r[0].name == name and r[1] == Relationship.PARENT:
                yield r[2].name

And we can now update `Research` to call the interface instead of the implementation:

In [7]:
class Research:
    def __init__(self, browser):
        for p in browser.find_all_children_of("John"):
            print(f'John has a child called {p}')

Note that the `browser` argument is for a `Relationships` instance, because `Relationships` inherits from `RelationshipBrowser`, which is an abstract class and thus cannot be instantiated.

We can test our new solution by running the exact same code as before:

In [8]:
parent = Person('John')
child1 = Person('Chris')
child2 = Person('Matt')

relationships = Relationships()
relationships.add_parent_and_child(parent, child1)
relationships.add_parent_and_child(parent, child2)

Research(relationships)

John has a child called Chris
John has a child called Matt
